<a href="https://colab.research.google.com/github/hxk271/SocDataSci/blob/main/archive/W10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 10 (자료의 전처리 II)

자료 전처리(data preprocessing)를 오늘도 계속 연습한다. 특히 오늘은 자료의 **집계(aggregation)**, **결합(merging)**, 그리고 <b>재배열(reshaping)</b>에 관해 공부한다.

본격적인 연습을 위해 다음 링크에서 오늘 사용할 파일들을 모두 준비하자.

In [ ]:
import gdown
links = ['https://drive.google.com/uc?id=1bXEgep5eQ__R2kOIHD5x2hP57IOAtktV',          #niaa2009
         'https://drive.google.com/uc?id=1bYQAy0y4zMGxWRZ4jPvv2BLkMIunuuG-',          #niaaa-report
         'https://drive.google.com/uc?id=1bZC-t1hm5ybla7gEr44UiS4K8ILS0l1B',          #population
         'https://drive.google.com/uc?id=1bYVuWXqOhb_32ExO2tTTrS_lqKnLBNUp',          #division
         'https://drive.google.com/uc?id=1hyYA9GyOeRryug08f1h4Gb85lcO_PbLG',          #german-credit
         'https://drive.google.com/uc?id=1csKImQToC6_JTRbyan9aJuDq9UYHw4tr']          #uber

for link in links:
    gdown.download(link)

## 1. 집계

> 집계가 무엇인지 다시 한 번 기억을 떠올려보자. 투표 결과의 집계를 연상하면 좋다. 집계하기 위해서는 집계할 단위별로 인덱스(index)를 설정하면 편리하다. 엑셀에서 `niaa-report.csv`를 불러와 어떤 식으로 인덱스를 설정하면 좋을지 미리 살펴보자.

In [ ]:
import pandas as pd

#alco = pd.read_csv("niaaa-report.csv", index_col = "State")            #왜 이것은 별로일까?
alco = pd.read_csv("niaaa-report.csv", index_col = ['State', 'Year'])
alco

> 일단 수열이나 데이터프레임이 주어진다면, <code>max()</code>, <code>min()</code>, <code>mean()</code>, <code>sum()</code> 등으로 기초통계량을 쉽게 계산할 수 있다는 사실을 기억하자. 이때 `axis=0`를 사용하면 변수 **간(across)** 집계를 수행할 수 있고, `axis=1`를 사용하면 변수 **내(within)** 집계를 수행할 수 있었다. 아래 명령어를 하나하나 연습해보자.


In [ ]:
#최대/최소
alco.max(axis = 1)
alco['Wine'].min()

#평균
alco.mean(axis = 1)
alco.median()

#합계
alco.sum()
alco.sum(axis = 1)

> 이런 계산을 전체 또는 그룹별로 수행하면 곧 <b>집계(aggregation)</b>가 된다. 집계는 대단히 중요한데 의외로 제대로 할 줄 모르는 경우가 많다(득표수는 무슨 종류의 집계인지 곰곰이 생각해보자).
>
>  데이터프레임에서 `groupby()`를 매서드를 활용해야 한다. 괄호 안에는 어떤 변수 단위로 집계할지 넣어준다.

In [ ]:
mean_alco = alco.groupby("Year").mean()
mean_alco

#시각화
mean_alco.plot()

> **연습문제 1-1**. `niaaa-report.csv`에서 맥주 총소비량을 주별로(by state) 집계하시오.

In [ ]:
alco.groupby('State')['Beer'].sum()

> **연습문제 1-2**. `niaaa-report.csv`에서 세 종류 주류의 총소비량을 연도별로(by year) 집계하시오.

In [ ]:
alco['alc'] = alco['Beer'] + alco['Wine'] + alco['Spirits']
alco.groupby('Year')['alc'].sum()
alco

#시각화
alco.groupby('Year')['alc'].sum().plot()

> 조금 어려울 수도 있긴 하지만, `agg()`를 사용하면 필요한 항목들을 한번에 계산할 수 있다. 이때 `agg`는 말할 필요도 없이 aggregate, 즉 집계를 뜻한다.

In [ ]:
alco
#주별-연도별로(by state-year) Wine은 평균 집계, Spirits은 최대값 집계
alco.groupby(['State','Year'])[['Wine','Spirits']].agg({'Wine' : 'mean', 'Spirits' : 'max'})

> **연습문제 1-3**. 다음 링크의 `penguins.csv`를 활용하여 펭귄의 종별로 부리(bill), 날개(flipper), 몸무게 평균을 계산하시오.
>
> ---
> https://drive.google.com/uc?id=1hpr7M5QmNCUgDbLd-Y8OnnaBZNWZpc-r

In [ ]:
df = pd.read_csv('penguins.csv')
df.groupby('species')[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']].mean()

> **연습문제 1-4**. 다음 링크의 `titanic.csv`를 활용하여 성별-좌석등급별 생존율을 계산하시오.
>
> ---
> https://drive.google.com/uc?id=1hsBgmMLiky5sXUSy-jt3ESsqNNyMERyr

In [ ]:
df = pd.read_csv('titanic.csv')

#성별-좌석등급별 생존율
df.groupby(['sex','class'])['survived'].mean()

## 2. 결합

> 두 개 이상의 자료가 따로따로 떨어져 있을 때보다, 하나로 합쳐졌을 때 훨씬 큰 잠재력을 발휘하곤 한다. 그 사례를 잘 생각해보는 것이 무엇보다 중요하다. 가령 편의점의 위치 자료는 성별-연령대별 인구 자료와 결합했을 때 훨씬 쓸모있다(Why?). >
> 자료를 합치기 위해서는 먼저 합칠 두 자료를 각각 불러오고, <b>공통 식별자(common identifier)</b>를 찾아야 한다. 아래 두 데이터를 불러와 각각의 내용을 살펴보자.

In [ ]:
drink = pd.read_csv("niaaa-report2009.csv")
drink

pop = pd.read_csv("population.csv")
pop

> 만약 연도별 혹은 주별로 1인당 맥주 소비량을 계산하려면 어떻게 해야할까? 일단 하나의 자료만으로는 그런 계산을 수행할 수 없다(Why?). 반드시 두 자료를 결합해야만 한다.
>
> `pd.merge()`로 두 자료를 결합할 수 있다. 상당히 중요한 스킬이므로 잘 이해할 필요가 있다.

In [ ]:
df = pd.merge(drink, pop, left_on = "State", right_on = "State")       #왼쪽 오른쪽 각각의 공통 변수
df

> 만약 <b>공통된 인덱스(common index)</b>가 있다면 더 쉽게 결합할 수 있다! 만일 다른 변수가 모두 숫자라면 나중에 계산을 편리하게 하기 위해 고유한 문자열(e.g., 주 이름)을 인덱스로 삼을 수도 있다.

In [ ]:
drink = pd.read_csv("niaaa-report2009.csv")
drink = drink.set_index("State")
drink

pop = pd.read_csv("population.csv", index_col = "State")
pop

> 이때는 `pd.merge()`를 다른 패러미터와 함께 사용한다.

In [ ]:
df = pd.merge(drink, pop, left_index = True, right_index = True)
df

> 위에서 설명한 둘 중 하나의 방식으로 자료를 결합할 수 있다. 물론 혼합할 수도 있다. 이제 미국 주들의 인구 십만 명당 맥주 소비량을 계산해보자.

In [ ]:
df['beer_per_capita'] = df['Beer'] / df['Population']
df

> 안된다. 왜 그럴까? 사실 성급하게 결합하려고 했던 점에서 문제가 예견되었다. 사실 합치기 전에 자료를 먼저 꼼꼼하게 살펴보아야 한다! `pop` 자료를 먼저 꼼꼼히 살펴보고 인구별로 **정렬(sort)**도 해보자.

In [ ]:
df.sort_values("Population")     #Population 변수별로 정렬

> 위의 결과가 이상하다. 하지만 잘 들여다보면 말이 되긴 한다. 근본적인 이유는 자료유형(dtype) 때문이다!

In [ ]:
df.info()

> `Population`이 <b>객체(object)</b>로 설정되어 있으므로 숫자 계산에서 오류가 나타났다! 우리는 이 문제에 대응하는 방법을 이미 배웠다. 또다른 방법은 애시당초 자료를 불러올때부터 문제의 원인을 지목하는 것이다!

In [ ]:
pop = pd.read_csv("population.csv", index_col="State", thousands = ',')
pop.info()

pop.sort_values("Population", ascending = True)

> 자 이제 두 데이터를 다시 합치고, 미국 주들의 인구 십만 명당 맥주 소비량을 계산해보자.

In [ ]:
df = pd.merge(drink, pop, left_on = "State", right_on = "State")

df['beer_per_capita'] = df['Beer'] / (df['Population'] / 100000)
df.sort_values('beer_per_capita', ascending = False)

> **연습문제 2-1**. `niaaa-report2009.csv`와 `division.csv`를 재주껏 결합하시오.

In [ ]:
import pandas as pd

alco2009 = pd.read_csv("niaaa-report2009.csv")
alco2009

division = pd.read_csv("division.csv")
division

df = pd.merge(alco2009, division, left_on = 'State', right_on = 'state')
df

> `pd.merge()` 이외의 방식도 가능하다. 각각 쓰임새가 조금씩 다르기 때문에 모두 익숙해져야 한다. 가령 `pd.concat()`도 자주 쓰인다. 참고로 `concat`은 <b>결합(concatenate)</b>을 의미한다. 이때는 인덱스가 공통적으로 설정되어 있어야 한다.

In [ ]:
pop = pd.read_csv("population.csv", index_col="State", thousands = ',')
drink = pd.read_csv("niaaa-report2009.csv", index_col = "State")

df = pd.concat([pop, drink], axis = 1)
df

> `pd.concat()`을 사용하여 결합(merging)이 아니라, <b>추가(appending)</b>할 수도 있다. 이건 두 자료를 좌우로 합치는 게 아니라 위아래로 합치는 것이고 상대적인 활용 빈도는 낮다.

In [ ]:
df = pd.concat([pop, drink], axis = 0)
df

> 두 개 이상의 수열이 합쳐지면 결국 하나의 행렬, 즉 자료가 된다. 그러므로 `pd.concat()`을 사용하여 수열을 결합(merging)한다면 하나의 데이터프레임으로 만들 수 있다!

> **연습문제 2-2**. 아래 연도(`year`), 실업률(`unemployment`), 인플레이션율(`inflation`) 수열을 하나로 합친 데이터프레임을 만드시오.
---
```python
year = pd.Series(range(2015, 2025))
unemployment = pd.Series([3.7, 3.8, 3.8, 3.9, 3.8, 4.0, 3.6, 2.9, 2.7, 2.8])
inflation = pd.Series([2.2, 1.6, 1.5, 1.2, 0.9, 0.7, 1.8, 4.1, 4.0, 2.1])
```

In [ ]:
year = pd.Series(range(2015, 2025))
unemployment = pd.Series([3.7, 3.8, 3.8, 3.9, 3.8, 4.0, 3.6, 2.9, 2.7, 2.8])
inflation = pd.Series([2.2, 1.6, 1.5, 1.2, 0.9, 0.7, 1.8, 4.1, 4.0, 2.1])

df = pd.concat([year, inflation, unemployment], axis = 1)     #axis=0도 실험해보자
df.columns = ["year", "inflation", "unemployment"]
df

> 참고로 지난 주에 배웠던 데이터프레임 만드는 방식을 다시 복습해 보자.

In [ ]:
year = pd.Series(range(2015, 2025))
unemployment = pd.Series([3.7, 3.8, 3.8, 3.9, 3.8, 4.0, 3.6, 2.9, 2.7, 2.8])
inflation = pd.Series([2.2, 1.6, 1.5, 1.2, 0.9, 0.7, 1.8, 4.1, 4.0, 2.1])

data = {"연도": year,
        "인플레이션율" : inflation,
        "실업률": unemployment}
df = pd.DataFrame(data)
df

## 3. 재배열

> 분석상 필요에 따라 자료의 꼴을 <b>재배열(reshaping)</b>해야할 때가 있다. 재배열도 자료 전처리 기법 가운데 만만치않게 까다로운 스킬이다. 여러 가지 방법으로 자료를 재배열할 수 있지만(즉 다 배워야 한다!), 일단 인덱스를 먼저 설정해놓고 `stack()`과 `unstack()`을 사용하는 편을 추천한다.

In [ ]:
alco2009 = pd.read_csv("niaaa-report2009.csv", index_col="State")
alco2009

> <b>넓은 꼴(wide form</b>에서 <b>긴 꼴(long form)</b>로 자료를 바꾸어보자. 주(`state`)가 인덱스로 설정된 상황이다. `stack()` 매서드를 통해 넓은 꼴로 주어진 자료를 쌓아 길게 만들어줄 수 있다. 이 매서드는 미리 설정된 인덱스를 고유한 아이디로 인식하고, 나머지 변수를 <b>키(key)</b>로, 나머지 관찰값들을 <b>값(value)</b>로 처리한다.

In [ ]:
wide2long = alco2009.stack()
wide2long

> 그리고나서 `unstack()`으로 다시 원래 형태, 즉 넓은 꼴로 되돌아갈 수 있다.

In [ ]:
wide2long.unstack()

> **연습문제 3-1**. 다음 `myjson`을 데이터프레임으로 변환하고, 이 자료를 이름(`name`)별로 긴 꼴 변환하시오.
---
```python
rec1 = {"id": 1001,
      "name": "김전일",
      "motto" : "할아버지의 이름을 걸고"}
rec2 = {"id": 1002,
      "name": "코난",
      "motto" : "진실은 언제나 하나"}
rec3 = {"id" : 1003,
      "name" : "김현우",
      "motto" : "할많하않"}
data = [rec1, rec2, rec3]      
```

In [ ]:
rec1 = {"id": 1001,
      "name": "김전일",
      "motto" : "할아버지의 이름을 걸고"}
rec2 = {"id": 1002,
      "name": "코난",
      "motto" : "진실은 언제나 하나"}
rec3 = {"id" : 1003,
      "name" : "김현우",
      "motto" : "할많하않"}
data = [rec1, rec2, rec3]

df = pd.DataFrame(data)
df
df = df.set_index('name')
df
df.stack()

> pandas에서는 보다 어렵고 복잡한 자료 재배열을 효율적으로 수행하기 위해 여러 명령어를 지원한다. 본격적인 실무나 연구에서 사용하려면 (제일 꼼꼼하게 작동하는) `pivot()`이 유용할 수도 있다. 이 매서드로 자료의 꼴을 재배열할 때는 (1) 새로운 행과 열이 무엇이 되어야 하는지를 우선 지정하고, (2) 각각의 행렬 안에 들어갈 원소를 지정하는 방식을 취한다.

In [ ]:
alco = pd.read_csv("niaaa-report.csv")
alco

alco.pivot(index = "Year", columns = "State", values = "Wine")

> **연습문제 3-2**. `niaaa-report.csv`에서 주별-연도별로 주류 총소비량을 구하여, 이를 주별(행)-연도별(열)로 정리하시오. 이때 각 셀에는 주류 총소비량이 표시되어야 한다.

In [ ]:
alco = pd.read_csv("niaaa-report.csv")
alco['max'] = alco[['Wine', 'Beer', 'Spirits']].sum(axis=1)
alco.pivot(index = 'State', columns = 'Year', values = 'max')

> **연습문제 3-3**. 생물학에서 <b>성적이형(sexual dimorphism)</b>이란 생식 경쟁으로 인해 발생하는 암수의 특성 차이를 의미한다. 다음 링크의 `penguins.csv`를 활용하여 펭균의 종별(Adelie, Chinstrap, Gentoo)로 수컷-암컷 간 평균 몸무게 차이를 출력하시오. 이때 출력해야 하는 변수는 종별 암컷 평균, 수컷 평균, 그리고 차이 평균이다. 이 표를 토대로 어떤 종에서 암수 몸무게 차이가 가장 큰지 식별하시오.
>
> ---
> https://drive.google.com/uc?id=1hpr7M5QmNCUgDbLd-Y8OnnaBZNWZpc-r

In [ ]:
df = pd.read_csv('penguins.csv')
bm = df.groupby(['species', 'sex'])['body_mass_g'].mean()
bm2 = bm.unstack()
bm2['mass_diff'] = bm2['Male'] - bm2['Female']
bm2

## 4. 재부호화

> 자료 내용 자체를 건드릴 일은 흔하지 않을 거라고 생각하기 쉽다. 그러나 연구 분야에 따라서는 자료를 수정하거나 <b>재부호화(recoding)</b>하는 경우가 매우 잦다. 이때 특히 `replace()`를 활용할 때가 많다.
>
> 어떤 독일 신용카드 정보에 담긴 나이, 직업, 성별, 주택 보유 여부, 만기, 부채 목적, 부채액수 등을 담은 자료를 불러오자. 여기서 성별의 빈도분포표를 확인해보자.

In [ ]:
#자료 불러오기
df = pd.read_csv('German_credit.csv')
df

#성별 빈도분포표
df['Sex'].value_counts()

> <b>범주형 변수(categorical variables)</b>를 분석하려면 때때로 <b>가변수(dummy variables)</b>로 <b>가부호화(dummy coding)</b>해야 할 필요가 있다. 만일 `female`을 1로, `male`을 0으로 바꾸고 싶다면 어떻게 할까? `replace()` 매서드를 사용한다. 다만 pandas의 과거 버전에서는 이런 경우 `object`에서 `int64`로 <b>다운캐스팅(downcasting)</b>을 해주었지만, 더이상 이런 편의는 제공하지 않는다.

In [ ]:
changes = {'male': 0, 'female': 1}
df['gender'] = df['Sex'].replace(changes)
df.info()

> 남(`0`)/녀(`1`)같은 두 개의 범주를 벗어나, 만일 두 개 이상의 범주가 있다면 가부호화를 훨씬 쉽게 할 수도 있다. 바로 `pd.get_dummies()` 함수를 사용한다!

In [ ]:
pd.get_dummies(df['Housing'])

> 이때는 0 대신 `False`, 1 대신 `True`를 반환하는 것이 특징이다. 새로 생겨야 하는 변수는 3개이므로 새로운 컬럼 이름도 3개를 지정해야 한다(Why?).

In [ ]:
df[['free', 'own', 'rent']] = pd.get_dummies(df['Housing'])
df

> 아니면 아예 `pd.concat()`을 사용해 결합한다!

In [ ]:
df = pd.read_csv('German_credit.csv')
tba = pd.get_dummies(df['Housing'])
df = pd.concat([df, tba], axis = 1)
df

> 다음으로 이번엔 연속변수인 나이(`Age`)를 구간별 자료로 바꾸어 보자. 이번엔 `cut()`을 사용해보자.

In [ ]:
df['Age'].describe()

agecat = pd.cut(df['Age'], bins=5)
agecat
agecat.value_counts()

> 이때 `bins=5`같은 옵션은 정확히 구간을 나눌때 불편할 수 있다(실제로 나뉜 구간이 보기 흉하다). 이 경우 리스트(list)를 옵션으로 제공하면 된다. 10부터 20까지, ..., 70부터 80까지 나타낸다.
>
> 단 결과를 주의해서 살펴보자! 이때 `()`는 <b>개구간(open interval)</b>로 초과/미만을, `[]`는 **폐구간(close interval)</b>로 이상/이하를 뜻한다.

In [ ]:
agecat = pd.cut(df['Age'], bins=[10, 20, 30, 40, 50, 60, 70, 80])
agecat

# 초과에서 이하까지
agecat.value_counts()               #sort by frequencies
agecat.value_counts(sort=False)     #do not sort!

# 이상에서 미만까지
agecat = pd.cut(df['Age'], bins= [10, 20, 30, 40, 50, 60, 70, 80], right=False)
agecat.value_counts(sort=False)

> **연습문제 4-1**. `division.csv`와 `niaaa-report.csv`를 결합하고 지역(`division`)을 범주형 변수로 재부호화하시오.

In [ ]:
division = pd.read_csv("division.csv")
tba = pd.get_dummies(division['region'])
df1 = pd.concat([division, tba], axis =1)

alco = pd.read_csv("niaaa-report.csv")
df2 = pd.merge(alco, df1, left_on='State', right_on='state')
df2

> **연습문제 4-2**. `German_credit.csv`에서 부채액(`Credit amount`)을 십분위수별로 나누고 그 범주값을 원자료에 새로 결합하시오.

In [ ]:
df = pd.read_csv('German_credit.csv')
tba = pd.cut(df['Credit amount'], bins=10)
pd.concat([df, tba], axis = 1)

## 5. 이상치

> <b>이상치(outlier)</b>는 다른 관측치들과 동떨어진 값이기 때문에 분석 결과를 왜곡할 가능성이 큰 관측치를 의미한다. 사회조사 자료에서 소득 같은 변수는 종종 극단값이 존재하기 때문에, 단순 평균만을 사용하면 전체 분포의 중심을 잘못 판단할 수 있다.
>
> 가장 기초적으로 $z$ 값에 기반하여 이상치 탐지를 수행해보자. 이때 $z$ 값은 각 관측치($X$)가 평균($\mu$)으로부터 얼마나 떨어져 있는지를 표준편차($\sigma$) 단위로 변환한 것이다(Why?).
>
> $$z = \dfrac{X-\mu}{\sigma}$$
>
> 절대값 기준으로 $z$의 절대값이 특정 수치, 가령 2나 3을 초과하는 경우, 통계적으로 드문 값으로 간주하여 이상치로 나타낼 수 있다(Why?).

In [ ]:
import numpy as np

np.log(df['Credit amount']).hist()                                                                 #로그변환된 다음 히스토그램
df['zscore'] = (df['Credit amount'] - df['Credit amount'].mean()) / df['Credit amount'].std()      #std(ddof=0)는 모표준편차
df['is_outlier'] = (df['zscore'] > 3) | (df['zscore'] < -3)                                        #근데 -3은 큰 의미가 없다(Why?)
df[df['is_outlier'] == True].value_counts()

> 참고로 이렇게 $z$ 값을 쉽게 계산할 수도 있다.

In [ ]:
from scipy import stats

df['zscore2'] = stats.zscore(df['Credit amount'])         #단 모표준편차로 계산함
df

>  z-score는 분포가 정규분포에 가까울 때 효과적이다. 실제 분석에서는 이상치를 무조건 제거하기보다, 그것이 입력 오류인지 아니면 중요한 사회적 현상을 반영하는 값인지를 먼저 해석하는 과정이 필요하다.
>
> 사실 데이터 사이언스 분야에서 이상치에 관한 연구는 그 자체로 큰 분야이다. 각종 신용카드, 보험, 대출 등의 사기 탐지(fraud detection) 분야 등에서 아주 많은 수요가 있기 때문이다. 사회보장, 테러, 범죄 등의 분야에서도 사용되며, 가령 <b>자살위기 개입(suicidal crisis intervention)</b> 등에서도 관련 연구가 이루어지고 있다.

> **연습문제 5-1**. `Duration`에 이상치가 있는지 여부를 확인하고 (만약 있다면) 전처리하시오.

In [ ]:
np.log(df['Duration']).hist()

df['dur_z'] = (df['Duration'] - df['Duration'].mean()) / df['Duration'].std()
df['outlier'] = df['dur_z'] > 3
df[df['outlier'] == True].value_counts()

#outlier=False인 것만 건지기
df2 = df[df['outlier'] == False]
df2
df2 = df2.drop(['dur_z', 'outlier'], axis=1)

## 6. 문자열의 처리

> 데이터 분석에서 문자열(string) 변수는 설문 응답, 리뷰, 댓글 등 사회과학 데이터에서 자주 등장하지만, 그대로 사용할 수 있는 경우는 드물다. "좋아요👍"나 "별로임..." 같은 응답에는 불필요한 특수문자나 이모티콘이 섞여 있어 분석이 곤란하다. 이런 상황에서 <b>정규표현식(regular expression)</b>은 텍스트 속에서 우리가 원하는 패턴을 찾아내거나 제거하는 데 강력한 도구가 된다.
>
> 우선 예제 데이터를 만들어보자.

In [ ]:
import pandas as pd

# 의류/의상 제품 리뷰 데이터
data = {'gender': [' Male ', 'FEMALE', 'female', ' male', 'F', 'M', 'm'],
        'comment': ['좋아요!!!', '별로임..', '가격대비굿', 'ㅎㅎㅎㅎ', '최고👍', 'no comment', '쵝오good~'],
        'income': [250, 320, 5000, 400, 290, 310, 100],
        'weight': ['60kg', '55kg', '53', '58', '63kg', '62kg', '100kg'],
        'age': [22, 25, 45, 34, 28, 39, 40]}
df = pd.DataFrame(data)
df

> 잘 살펴보면 `comment`는 문자열이다. 이제부터 이걸 적당히 전처리해보자.
>
> `str.replace()` 메서드에 정규표현식을 적용하면 문자를 정제(cleaning)할 수 있다. 아례 예제 코드에서는 `r'[^가-힣 ]'`라는 패턴을 사용해 한글과 공백을 제외한 모든 문자를 제거한다. 이를 통해 분석에 방해가 되는 기호나 이모티콘을 효과적으로 없앨 수 있다. 여기서 `^`는 **not**을 의미한다.[...]: 문자 집합을 의미한다. 그리고 `[]`는 대괄호 안의 문자 중 하나와 일치하는 경우를 의미한다.

In [ ]:
#한글만 남기기, 공백 제거, 영어나 특수 문자 제거
df['comment_clean'] = df['comment'].str.replace(r'[^가-힣 ]', '', regex=True)
df

#패러미터 구조를 살펴보자
? df.replace

> 정규표현식은 생각보다 다양하게 쓰인다. 예를 들어, 거대한 텍스트 덩어리(corpus) 속에서 hxk271@cbnu.ac.kr 같은 이메일 주소만을 파싱(parsing)하고 싶다면 어떻게 할 수 있을까? 머리 속으로 잘 생각해보자.
>
> 답만 말하자면, `\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b`와 같은 패턴으로 구성할 수 있다. 각 요소는 사용자 이름, `@` 기호, 도메인 이름, `.`, 그리고 최상위 도메인(top-level domain; TLD)을 의미한다. 이메일 정규식의 주요 구성 요소를 하나하나 살펴보자.
> ```
> `\b`: 단어의 처음과 끝 경계를 나타내어 이메일 주소가 다른 단어에 포함되지 않도록 한다.
>
> `[A-Za-z0-9._%+-]+`: 사용자 이름 부분. 알파벳, 숫자, 밑줄, 점, 퍼센트, 플러스, 하이픈 문자를 포함하며, 한 번 이상 반복될 수 있다(`+`)
>
> `@`: `@` 기호에 정확히 일치시킨다.
>
> `[A-Za-z0-9.-]+`: 도메인 이름을 일치시킨다. 알파벳, 숫자, 점, 하이픈 문자를 포함하며, 한 번 이상 반복될 수 있다.
>
> `\.`: `.`에 일치시킨다.
>
> `[A-Z|a-z]{2,}`: 최상위 도메인(TLD)을 일치시시킨다. 최소 두 자리 이상의 알파벳으로 구성된다.
> ```

In [ ]:
import re         #정규표현식 전용 라이브러리

text = """
교수 소개
학과소개 교수 소개
전임교수
강사
명예교수
*교수 이름 클릭 시 세부정보 확인 가능
이항우 교수 more
전공분야
정보사회학, 문화사회학, 정치사회학
연락처
043-261-2181
메일
hwyi@chungbuk.ac.kr
이해진 교수 more
전공분야
사회운동/시민사회, 지역사회학, 농식품사회학, 사회적경제
연락처
043-261-2182
메일
jinlee@chungbuk.ac.kr
박정미 부교수 more
전공분야
젠더와 성의 사회학, 역사사회학, 정책과 사회운동
연락처
043-261-2183
메일
parkjm@chungbuk.ac.kr
홍덕화 부교수 more
전공분야
환경사회학, 과학기술사회학, 도시사회학
연락처
043-261-2184
메일
dhhong@chungbuk.ac.kr
서선영 조교수 more
전공분야
이주학(Migration Studies), 노동/인권 연구, 질적연구방법론
연락처
043-261-2211
메일
seonyoungseo@chungbuk.ac.kr
김현우 부교수 more
전공분야
집단행동/사회운동, 조직사회학, 사회통계, 계산사회과학
연락처
043-261-2186
메일
hxk271@chungbuk.ac.kr
"""

#이메일을 위한 정규표현식
email_regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"

#모든 이메일 주소 찾기(re.findall)
emails = re.findall(email_regex, text)
emails

> **연습문제 6-1**. `ㅎㅎㅎㅎ`를 지우지 않고 살리기 위해서는 어떻게 변경해야 할까?

In [ ]:
#한글만 남기기, 공백 제거, 영어나 특수 문자 제거
df['comment_clean'] = df['comment'].str.replace(r'[^ㄱ-힣 ]', '', regex=True)
df

> 문자열 정제 이후에는 텍스트 속에서 특정 단어나 감정 표현을 탐지할 수도 있다. 예제의 `str.extract()` 구문은 “좋”, “굿”, “최고” 등의 단어가 포함되어 있는지를 찾아내며, 그 결과를 True/False 형태의 변수(`comment_sentiment`)로 변환한다.

In [ ]:
#extract 매서드로 좋은 글자만을 식별(|은 or를 뜻한다)
df['sentiment'] = df['comment'].str.extract(r'(좋|굿|최고)')
df

#결측치 메꾸는 동시에 True/False로 변경
df['sentiment'] = df['sentiment'].notnull()
df

> **연습문제 6-2**. 위 데이터프레임에서 `weight` 변수의 단위를 'kg'으로 통일하되 정수로 표현하시오.> (3) 연령대를 나타내는 'age_group' 변수를 새로 만드시오.

In [ ]:
df['weight_clean'] = df['weight'].str.replace('kg', '', regex=False)
df.info()

df['weight_clean'] = pd.to_numeric(df['weight_clean'])
df.info()

## 7. 자료 전처리 연습

> 자료 전처리를 실제로 처음부터 끝까지 제대로 수행해보자. 이번에 사용할 자료 `Uber.csv`는 미국, 스리랑카, 파키스탄의 2016년 1월부터 12월까지 우버 운행 기록이다.
>
> ---
> ```python
>import gdown
>link = 'https://drive.google.com/uc?id=1csKImQToC6_JTRbyan9aJuDq9UYHw4tr'
>gdown.download(link)
>```

In [ ]:
df = pd.read_csv('Uber.csv')
df

> 다양한 변수가 있는데 먼저 훑어보는 것도 중요하다!

In [ ]:
df.info()
df.describe()
df['CATEGORY'].value_counts()
df['PURPOSE'].value_counts()
df['START'].unique()
df['STOP'].unique()

> 앞서 `pd.to_datetime()`을 통해 object를 날짜 속성으로 바꿀 수 있음을 배웠다. 출발과 도착 시간을 먼저 전처리하자!
>
> 가공하고 살펴보면 편리하게도 날짜와 시간 출력 형식이 모두 일관되도록 수정되었다. 또 예측치 못한 에러(error)가 나오더라도 무시하고 강제로 건너뛰었다(Why?).

In [ ]:
df['start'] = pd.to_datetime(df['START_DATE'], errors='coerce')
df['end'] = pd.to_datetime(df['END_DATE'], errors='coerce')
df

> `sort_values()`를 사용한다면 자료를 정렬(sort)할 수도 있다.

In [ ]:
df.sort_values(['start','end'])

> 우버 사용 시간이 어떻게 되는지 계산해보자.

In [ ]:
# datetime 변수의 단위 변환 (분)
df['duration'] = (df['end'] - df['start'])
df

#확인
df['duration'].sort_values()

> 주행시간을 살펴보니 벌써 **이상치(outliers)** 3개가 눈에 들어온다. 약간 느슨하게 판단하긴 했지만, 어쨋든 이를 삭제하자.

In [ ]:
outlier_idx = [269, 776, 1155]
df.loc[outlier_idx]

#drop()으로 삭제
df = df.drop(outlier_idx, axis = 0)

#재확인
df['duration'].sort_values()

> **연습문제 7-1**. `uber.csv`에서 `CATEGORY`와 `PURPOSE` 별로 이용시간(`duration`)을 평균 집계하여 비교해보자.

In [ ]:
df.groupby(['CATEGORY', 'PURPOSE'])['duration'].mean()

> **연습문제 7-2**. `Uber.csv`를 재배열(reshaping)하여 출발지점(`STOP`)을 행(index)으로, 도착지점(`STOP`)을 열(columns)로 하고, 각 셀(values)에는 평균 이용시간(`duration`)을 넣는 행렬을 만드시오.

In [ ]:
df2 = df.groupby(['START', 'STOP'])['duration'].mean()          #먼저 평균값을 구해주지 않으면 중복으로 인해 재배열이 안됨!
df2 = df2.reset_index()                                         #수열인 상태에서는 작동 안한다고 했으므로!
df2.pivot(index='START', columns='STOP', values='duration')